In [ ]:
!pip install transformers
!pip install torch
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 78.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 93.3 M

Imports:

In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Set filepaths for text file and csv file:

In [ ]:
# text_filepath = "../home/schen9/dataset.txt"
text_filepath = "dataset/part_1.txt"
csv_filepath = "dataset.csv"

Read in the text file and split it by double newline:

In [ ]:
with open(text_filepath, "r", encoding="utf-8", errors="ignore") as text_file:
    rows = text_file.read().strip().split("\n\n")[:1000]

print(f"Finished reading from {text_filepath}") 

Write the text to the csv file:

In [ ]:
import csv

with open(csv_filepath, "w", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Text"]) # Header
    for i, row in enumerate(tqdm(rows)):
        writer.writerow([row.strip()])

print(f"Finished writing to {csv_filepath}") 

Load the csv onto a hugging face dataset object:

In [ ]:
from datasets import Dataset, load_dataset

dataset = load_dataset("csv",data_files=csv_filepath) 
dataset = dataset.filter(lambda x: x["Text"] is not None) # filter out NoneTypes
dataset

Load the tokenizer:

In [ ]:
from transformers import AutoTokenizer

transformer_name = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(transformer_name, use_fast=True)

Tokenize all the text in batches:

In [ ]:
def tokenize(batch):
    return tokenizer(batch["Text"], truncation=True)

batch_size = 50
tokens = dataset.map(tokenize, batched=True, batch_size=batch_size)
tokens

In [8]:
tokens.save_to_disk("my_tokens")

Saving the dataset (0/2 shards):   0%|          | 0/446203 [00:00<?, ? examples/s]

In [ ]:
df = tokens["train"].to_pandas()
df

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(transformer_name, output_hidden_states = True).to(device)
model.eval()

In [ ]:
def get_embeddings(t_vector, a_mask, model):
    token_tensor = torch.tensor([t_vector]).to(model.device)
    seg_tensor = torch.tensor([a_mask]).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=token_tensor, attention_mask=seg_tensor)
        hidden_state = outputs[1]
    token_embeddings = torch.stack(hidden_state, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs = []
    for token in token_embeddings:
    # "token" is a [12 x 768] tensor
    # sum the vectors from the last four layers
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs.append(sum_vec)
    return token_vecs

# Iterate over rows in DataFrame and call the function
token_counts = {i: 0 for i in range(tokenizer.vocab_size)}
avg_embeddings = {}

for t_vector, a_mask in tqdm(zip(df["input_ids"], df["attention_mask"]),total=len(df)):
    embeddings = get_embeddings(t_vector, a_mask, model)
    for t_index, token in enumerate(t_vector):
        token_counts[token] += 1
        embedding = embeddings[t_index] # Get the corresponding embedding for the token
        if token not in avg_embeddings:
            avg_embeddings[token] = embedding
        else:
            avg_embeddings[token] = torch.add(embedding, avg_embeddings[token])
    # break  # Break after first iteration for testing

# Average the embeddings
for token in avg_embeddings.keys():
    avg_embeddings[token] = torch.div(avg_embeddings[token], token_counts[token])

In [ ]:
import pickle 

with open('avg_embeddings.pkl', 'wb') as f:
    pickle.dump(avg_embeddings, f)